In [1]:
import numpy as np
import pandas as pd
from calendar import monthrange
from typing import Union, List, Dict
from libraries.IndexesExtraction import IndexesExtraction

In [2]:
periods = [str(i) for i in list(range(1964, 2022))]

In [3]:
classification_rules = {
    'fraca':            np.array(range(-31, -51, -1)),
    'moderada':         np.array(range(-51, -101, -1)),
    'intensa':          np.array(range(-101, -251, -1)),
    'super_intensa':    np.array(range(-251, -1001, -1)),
}

indexes = IndexesExtraction(periods=periods)
indexes.make_classification(classification_rules, classification_by_column='Dst_index')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24506    0 24394  100   112  24615    113 --:--:-- --:--:-- --:--:-- 24728
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25298    0 25186  100   112  30019    133 --:--:-- --:--:-- --:--:-- 30152
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24506    0 24394  100   112  30645    140 --:--:-- --:--:-- --:--:-- 30747
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24506    0 24394  100   112  30996    142 --:--:-- --:--:-- --:--:-- 31098
  % Total    % Received % Xferd  Average Speed   Tim

In [4]:
df_test = indexes.df.copy()

In [11]:
def save_processed_data(df, folder_name: str, name: str):
    processed_data = pd.HDFStore(path=f'/home/gabriel/tcc/OMNI/processed_data/{name}.h5')
    processed_data.append('df', df)
    processed_data.close()
    print(f'Seus dados foram salvos em: processed_data/{name}.h5\n')

save_processed_data(df=df_test, folder_name='OMNI', name='full_data')

Seus dados foram salvos em: processed_data/full_data.h5



In [ ]:
df_test

In [ ]:
# df_test.drop(df_test[df_test['Dst_index'] == 999.9].index, inplace=True)
# df_test.drop(df_test[df_test['B_scalar'] == 999.9].index, inplace=True)
# df_test.drop(df_test[df_test['Bz_GSM'] == 999.9].index, inplace=True)
df_test.drop(df_test[df_test['Dst_index'] == 999.9].index, inplace=True)

In [ ]:
def class_count(df):
    count = {"fraca": 0, "moderada": 0, "intensa": 0, "super_intensa": 0}
    for i in range(1, len(df)):
        count[df['classification'].iloc[i]]+=1
    print(count)

In [ ]:
class_count(df_test)